# Figure - Iteration time comparison


In [ ]:
import os
work_dir = "H:\workspace\ptyrad"
os.chdir(work_dir)
print("Current working dir: ", os.getcwd())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ptyrad.data_io import load_hdf5, load_pt, load_fields_from_mat
import h5py
from natsort import natsorted
from skimage.metrics import structural_similarity as ssim
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift


In [ ]:
def center_crop(image, crop_height, crop_width, offset = (0,0)):
    """
    Center crops a 2D or 3D array (e.g., an image).

    Args:
        image (numpy.ndarray): The input array to crop. Can be 2D (H, W) or 3D (H, W, C).
        crop_height (int): The desired height of the crop.
        crop_width (int): The desired width of the crop.

    Returns:
        numpy.ndarray: The cropped image.
    """
    if len(image.shape) not in [2, 3]:
        raise ValueError("Input image must be a 2D or 3D array.")

    height, width = image.shape[-2:]

    if crop_height > height or crop_width > width:
        raise ValueError("Crop size must be smaller than the input image size.")

    start_y = (height - crop_height) // 2 + offset[0]
    start_x = (width - crop_width) // 2 + offset[0]

    return image[..., start_y:start_y + crop_height, start_x:start_x + crop_width]

## Use SSIM with ground truth

In [ ]:
gt_path = "data/paper/simu_tBL_WSe2/phonon_temporal_spatial_N16384_dp128.hdf5"

# These two seems too noisy, orblur needs to be stronger, or we need to add sparsity
# dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_ozblur1_oathr0.98_opos_sng1.0_1e6/"
# dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_orblur0.2_ozblur1_oathr0.98_opos_sng1.0_1e6/"
# dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_orblur0.3_ozblur1_oathr0.98_opos_sng1.0_1e6/"

# All the below ones show good SSIM
## 0.918, smooth, good, no sparsity
# dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_orblur0.4_ozblur1_oathr0.98_opos_sng1.0_1e6/"

## 0.93, kink, very good
# dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_orblur0.4_ozblur1_oathr0.98_opos_sng1.0_spr0.01_1e6/"

## 0.924, smooth, very good
dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_orblur0.4_ozblur1_oathr0.98_opos_sng1.0_spr0.03_1e6/"

## 0.907, smooth, good but a bit overlapping
# dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_orblur0.3_ozblur1_oathr0.98_opos_sng1.0_spr0.03_1e6/"

## 0.91, wavy, okay, no positivity
# dir_ptyrad = "output/paper/simu_tBL_WSe2/20250219_ptyrad_convergence/full_N16384_dp128_flipT001_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr1e-4_orblur0.4_ozblur1_oathr0.98_sng1.0_1e6/"

dir_ptyshv = "data/paper/simu_tBL_WSe2/6/roi6_Ndp128_step128\MLs_L1_p12_g16_pc1_noModel_mm_Ns6_dz2_reg0.1"
dir_py4dstem = "output/paper/simu_tBL_WSe2/20250219_py4dstem_convergence/20250219_N16384_dp128_flipT001_random16_p12_6slice_dz2_update0.5_kzf0.1_1e6/"

# Get file names sorted
files_ptyrad = []
for file in os.listdir(dir_ptyrad):
    if file.startswith('model'):
        files_ptyrad.append(file)
files_ptyrad.sort()
print(f"Found {len(files_ptyrad)} ptyrad files")

files_ptyshv = []
for file in os.listdir(dir_ptyshv):
    if file.startswith('Niter'):
        files_ptyshv.append(file)
files_ptyshv = natsorted(files_ptyshv)
print(f"Found {len(files_ptyshv)} ptyshv files")
        
files_py4dstem = []
for file in os.listdir(dir_py4dstem):
    if file.startswith('model'):
        files_py4dstem.append(file)
files_py4dstem.sort()
print(f"Found {len(files_py4dstem)} py4dstem files")

In [ ]:
# Get ptyrad results
objs_ptyrad = []
for file in files_ptyrad:
    ckpt = load_pt(os.path.join(dir_ptyrad, file))
    obj = ckpt['optimizable_tensors']['objp'].detach().cpu().numpy().squeeze().sum(0)
    objs_ptyrad.append(obj)
iter_time_ptyrad = ckpt['avg_iter_t']

# Get ptyshv results
objs_ptyshv = []
for file in files_ptyshv:
    with h5py.File(os.path.join(dir_ptyshv, file), "r") as hdf_file:
        obj = np.angle(hdf_file['object'][()].view('complex128')).sum(0)
        obj = obj.T
        iter_time_ptyshv = hdf_file['outputs']['avgTimePerIter'][()].squeeze()[()]
        objs_ptyshv.append(obj)
        
# Get py4dstem results
objs_py4dstem = []
for file in files_py4dstem:
    with h5py.File(os.path.join(dir_py4dstem, file), "r") as hdf_file:
        obj = np.angle(hdf_file['object'][()].view('complex64')).sum(0)
        iter_time_py4dstem = hdf_file['iter_times'][()].mean(0)
        objs_py4dstem.append(obj)

In [ ]:
crop_height, crop_width = 170, 170 # Note that the window size and alignment would also mpact the SSIM value
# The image shifts are obtained via SIFT registration using imageJ

# Get images
gt_phase = center_crop(load_hdf5(gt_path, dataset_key = 'gt_phase').sum(0), crop_height, crop_width, offset = [1,1])

ssims_ptyrad = []
for img_ptyrad in objs_ptyrad:
    img_ptyrad = center_crop(img_ptyrad, crop_height, crop_width, offset = [0,0])
    # shift, error, diffphase = phase_cross_correlation(gt_phase, img_ptyrad, upsample_factor=100)
    shift = [-0.205,-0.2675]
    img_ptyrad = np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img_ptyrad), shift)))
    img_ptyrad -= img_ptyrad.min()
    
    ssim_ptyrad = ssim(img_ptyrad, gt_phase, data_range=(gt_phase.max() - gt_phase.min()))
    print(ssim_ptyrad)
    ssims_ptyrad.append(ssim_ptyrad)
    
ssims_ptyshv = []
for img_ptyshv in objs_ptyshv:
    img_ptyshv = center_crop(img_ptyshv, crop_height, crop_width, offset = [-1,-1])
    # shift, error, diffphase = phase_cross_correlation(gt_phase, img_ptyshv, upsample_factor=100)
    shift = [-0.35,-0.416]
    img_ptyshv = np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img_ptyshv), shift)))
    img_ptyshv -= img_ptyshv.min()
    
    ssim_ptyshv = ssim(img_ptyshv, gt_phase, data_range=(gt_phase.max() - gt_phase.min()))
    ssims_ptyshv.append(ssim_ptyshv)
        
ssims_py4dstem = []
for img_py4dstem in objs_py4dstem:
    img_py4dstem = center_crop(img_py4dstem, crop_height, crop_width, offset = [1,1])
    # shift, error, diffphase = phase_cross_correlation(gt_phase, img_py4dstem, upsample_factor=100)
    shift = [0.32, 0.22]
    img_py4dstem = np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img_py4dstem), shift)))
    img_py4dstem -= img_py4dstem.min()
    
    ssim_py4dstem = ssim(img_py4dstem, gt_phase, data_range=(gt_phase.max() - gt_phase.min()))
    ssims_py4dstem.append(ssim_py4dstem)

In [ ]:
# from tifffile import imwrite
# imwrite('ptyrad.tif', np.float32(objs_ptyrad))
# imwrite('ptyshv.tif', np.float32(objs_ptyshv))
# imwrite('py4dstem.tif', np.float32(objs_py4dstem))
# imwrite('stack.tif', np.stack([np.float32(gt_phase), np.float32(img_ptyrad), np.float32(img_ptyshv), np.float32(img_py4dstem)]).astype('float32'))

In [ ]:
iterations = np.array([i for i in range(1,20)] + [i for i in range(20,210,10)])

plt.figure()
plt.plot(iterations[:len(ssims_ptyrad)]*iter_time_ptyrad, ssims_ptyrad)
plt.plot(iterations[:len(ssims_ptyshv)]*iter_time_ptyshv, np.array(ssims_ptyshv))
plt.plot(iterations[:len(ssims_py4dstem)]*iter_time_py4dstem, np.array(ssims_py4dstem))
plt.xscale('log')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.ticker as tck

mpl.rc('xtick', direction='in')
mpl.rc('xtick.major', width=1, size=3.5)
mpl.rc('xtick.minor', width=1, size=2)
mpl.rc('ytick', direction='in')
mpl.rc('ytick.major', width=1, size=3.5)
mpl.rc('ytick.minor', width=1, size=2)

# Plot Data
# Note that these data are all computed on 20GB slice

iterations = np.array([i for i in range(1,20)] + [i for i in range(20,210,10)])

batch_sizes = np.array([16, 32, 64, 128, 256, 512, 1024])
time_ptyrad_batch = np.array([19.0, 16.45, 15.18, 14.37, 13.91, 13.84, 13.71])
time_ptyshv_batch = np.array([84.08, 42.31, 23.47, 17.18, 13.02, 11.87, 11.15])
time_py4dstem_batch = np.array([145.57, 75.85, 43.40, 23.20, 15.47, 12.32, 11.51])

probe_modes = np.array([1, 3, 6, 12])
time_ptyrad_modes = np.array([12.87, 14.18, 19.00, 26.46])
time_ptyshv_modes = np.array([19.07, 45.46, 84.08, 161.49])
time_py4dstem_modes = np.array([29.70, 74.71, 145.57, 282.91])

num_slices = np.array([1, 3, 6])
time_ptyrad_slices = np.array([7.09, 11.34, 19.00])
time_ptyshv_slices = np.array([22.24, 44.38, 84.08])
time_py4dstem_slices = np.array([24.26, 82.01, 145.57])

# Speedup factor
speedup_factor_iter = iter_time_py4dstem/iter_time_ptyrad
speedup_factor_batch = time_py4dstem_batch[0] / time_ptyrad_batch[0]
speedup_factor_modes = time_py4dstem_modes[-1] / time_ptyrad_modes[-1]
speedup_factor_slices = time_py4dstem_slices[-1] / time_ptyrad_slices[-1]

# Global font/line control
linewidth = 0.8
markersize = 3
fontsize_title = 9
fontsize_subtitle = 7
fontsize_label = 9
fontsize_legend = 5
annotate_size = 7

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(7, 5), dpi=300)

# Panel 1: Error vs. Iteration
axes[0, 0].plot(iterations[:len(ssims_ptyrad)]  *iter_time_ptyrad, ssims_ptyrad, label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize, zorder=10)
axes[0, 0].plot(iterations[:len(ssims_ptyshv)]  *iter_time_ptyshv, ssims_ptyshv, label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[0, 0].plot(iterations[:len(ssims_py4dstem)]*iter_time_py4dstem, ssims_py4dstem, label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)

axes[0, 0].set_title('SSIM vs. Reconstruction Time', fontsize=fontsize_title)
axes[0, 0].text(0.70, 0.20, '200 iterations, batch size 16 \n12 probes, 6 slices', 
                transform=axes[0, 0].transAxes, ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[0, 0].set_xlabel('Reconstruction Time (sec)', fontsize=fontsize_label)
axes[0, 0].set_ylabel('SSIM', fontsize=fontsize_label)
axes[0, 0].text(-0.2, 1.08, 'a', transform=axes[0, 0].transAxes, fontsize=16, fontweight='bold')  # Label "a"
# axes[0, 0].text(0.58, 0.725, f'{np.round(speedup_factor_iter,1)}x faster', transform=axes[0, 0].transAxes, fontsize=7, fontweight='bold', c='C0')  # Label "24x faster"
axes[0, 0].annotate(f'{iter_time_ptyrad:.0f} sec', (iter_time_ptyrad, ssims_ptyrad[0]), textcoords="offset points", xytext=(20,0), ha='center', color='C0', fontsize=annotate_size)
axes[0, 0].annotate(f'{iter_time_ptyshv:.0f} sec', (iter_time_ptyshv, ssims_ptyshv[0]), textcoords="offset points", xytext=(3,-10), ha='center', color='C1', fontsize=annotate_size)
axes[0, 0].annotate(f'{iter_time_py4dstem:.0f} sec', (iter_time_py4dstem, ssims_py4dstem[0]), textcoords="offset points", xytext=(18,-6), ha='center', color='C2', fontsize=annotate_size)

axes[0, 0].legend(fontsize=fontsize_legend)
# axes[0, 0].set_ylim(0.2,1.2)
axes[0, 0].set_xscale('log')
axes[0, 0].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Panel 2: Iter Time vs. Batch Sizes
axes[0, 1].plot(np.arange(len(batch_sizes)), time_ptyrad_batch, label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize)
axes[0, 1].plot(np.arange(len(batch_sizes)), time_ptyshv_batch, label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[0, 1].plot(np.arange(len(batch_sizes)), time_py4dstem_batch, label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)
axes[0, 1].set_title('Iteration Time vs. Batch Sizes', fontsize=fontsize_title)
axes[0, 1].text(0.5, 0.95, '6 probes, 6 slices', transform=axes[0, 1].transAxes, 
                ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[0, 1].set_xlabel('Batch Sizes', fontsize=fontsize_label)
axes[0, 1].set_ylabel('Iteration Time (sec)', fontsize=fontsize_label)
axes[0, 1].set_xticks(np.arange(len(batch_sizes)))
axes[0, 1].set_xticklabels([str(int(b)) for b in batch_sizes], fontsize=fontsize_label)
axes[0, 1].text(-0.1, 1.08, 'b', transform=axes[0, 1].transAxes, fontsize=16, fontweight='bold')  # Label "b"
# axes[0, 1].text(0.01, 0.08, f'{np.round(speedup_factor_batch,1)}x faster', transform=axes[0, 1].transAxes, fontsize=7, fontweight='bold', c='C0')  # Label "7.7x faster"
axes[0, 1].annotate(f'{time_ptyrad_batch[0]:.0f} sec', (0, time_ptyrad_batch[0]), textcoords="offset points", xytext=(10,5), ha='center', color='C0', fontsize=annotate_size)
axes[0, 1].annotate(f'{time_ptyshv_batch[0]:.0f} sec', (0, time_ptyshv_batch[0]), textcoords="offset points", xytext=(9,4), ha='center', color='C1', fontsize=annotate_size)
axes[0, 1].annotate(f'{time_py4dstem_batch[0]:.0f} sec', (0, time_py4dstem_batch[0]), textcoords="offset points", xytext=(10,5), ha='center', color='C2', fontsize=annotate_size)

axes[0, 1].legend(fontsize=fontsize_legend)
axes[0, 1].set_ylim(-10,165)
axes[0, 1].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Panel 3: Iter Time vs. Probe Modes
axes[1, 0].plot(probe_modes, time_ptyrad_modes, label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize)
axes[1, 0].plot(probe_modes, time_ptyshv_modes, label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[1, 0].plot(probe_modes, time_py4dstem_modes, label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)
axes[1, 0].set_title('Iteration Time vs. Probe Modes', fontsize=fontsize_title)
axes[1, 0].text(0.5, 0.95, 'Batch size 16, 6 slices', transform=axes[1, 0].transAxes, 
                ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[1, 0].set_xlabel('Number of Probe Modes', fontsize=fontsize_label)
axes[1, 0].set_ylabel('Iteration Time (sec)', fontsize=fontsize_label)
axes[1, 0].set_xticks(probe_modes)
axes[1, 0].set_xticklabels([str(int(p)) for p in probe_modes], fontsize=fontsize_label)
axes[1, 0].text(-0.2, 1.08, 'c', transform=axes[1, 0].transAxes, fontsize=16, fontweight='bold')  # Label "c"
# axes[1, 0].text(0.75, 0.15, f'{np.round(speedup_factor_modes,1)}x faster', transform=axes[1, 0].transAxes, fontsize=7, fontweight='bold', c='C0')  # Label "10.7x faster"
axes[1, 0].annotate(f'{time_ptyrad_modes[-2]:.0f} sec', (probe_modes[-2], time_ptyrad_modes[-2]), textcoords="offset points", xytext=(15,-7), ha='center', color='C0', fontsize=annotate_size)
axes[1, 0].annotate(f'{time_ptyshv_modes[-2]:.0f} sec', (probe_modes[-2], time_ptyshv_modes[-2]), textcoords="offset points", xytext=(14,-6), ha='center', color='C1', fontsize=annotate_size)
axes[1, 0].annotate(f'{time_py4dstem_modes[-2]:.0f} sec', (probe_modes[-2], time_py4dstem_modes[-2]), textcoords="offset points", xytext=(14,-9), ha='center', color='C2', fontsize=annotate_size)

axes[1, 0].annotate(f'{time_ptyrad_modes[-1]:.0f} sec', (probe_modes[-1], time_ptyrad_modes[-1]), textcoords="offset points", xytext=(-5,5), ha='center', color='C0', fontsize=annotate_size)
axes[1, 0].annotate(f'{time_ptyshv_modes[-1]:.0f} sec', (probe_modes[-1], time_ptyshv_modes[-1]), textcoords="offset points", xytext=(-5,4), ha='center', color='C1', fontsize=annotate_size)
axes[1, 0].annotate(f'{time_py4dstem_modes[-1]:.0f} sec', (probe_modes[-1], time_py4dstem_modes[-1]), textcoords="offset points", xytext=(-5,5), ha='center', color='C2', fontsize=annotate_size)

axes[1, 0].legend(fontsize=fontsize_legend)
axes[1, 0].set_ylim(-15,325)
axes[1, 0].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Panel 4: Iter Time vs. Slices
axes[1, 1].plot(num_slices, time_ptyrad_slices, label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize)
axes[1, 1].plot(num_slices, time_ptyshv_slices, label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[1, 1].plot(num_slices, time_py4dstem_slices, label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)
axes[1, 1].set_title('Iteration Time vs. Slices', fontsize=fontsize_title)
axes[1, 1].text(0.5, 0.95, 'Batch size 16, 6 probes', transform=axes[1, 1].transAxes, 
                ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[1, 1].set_xlabel('Number of Slices', fontsize=fontsize_label)
axes[1, 1].set_ylabel('Iteration Time (sec)', fontsize=fontsize_label)
axes[1, 1].set_xticks(num_slices)
axes[1, 1].set_xticklabels(num_slices, fontsize=fontsize_label)
axes[1, 1].text(-0.1, 1.08, 'd', transform=axes[1, 1].transAxes, fontsize=16, fontweight='bold')  # Label "d"

axes[1, 1].annotate(f'{time_ptyrad_slices[-2]:.0f} sec', (num_slices[-2], time_ptyrad_slices[-2]), textcoords="offset points", xytext=(15,-7), ha='center', color='C0', fontsize=annotate_size)
axes[1, 1].annotate(f'{time_ptyshv_slices[-2]:.0f} sec', (num_slices[-2], time_ptyshv_slices[-2]), textcoords="offset points", xytext=(14,-6), ha='center', color='C1', fontsize=annotate_size)
axes[1, 1].annotate(f'{time_py4dstem_slices[-2]:.0f} sec', (num_slices[-2], time_py4dstem_slices[-2]), textcoords="offset points", xytext=(14,-9), ha='center', color='C2', fontsize=annotate_size)

axes[1, 1].annotate(f'{time_ptyrad_slices[-1]:.0f} sec', (num_slices[-1], time_ptyrad_slices[-1]), textcoords="offset points", xytext=(-5,5), ha='center', color='C0', fontsize=annotate_size)
axes[1, 1].annotate(f'{time_ptyshv_slices[-1]:.0f} sec', (num_slices[-1], time_ptyshv_slices[-1]), textcoords="offset points", xytext=(-5,4), ha='center', color='C1', fontsize=annotate_size)
axes[1, 1].annotate(f'{time_py4dstem_slices[-1]:.0f} sec', (num_slices[-1], time_py4dstem_slices[-1]), textcoords="offset points", xytext=(-5,5), ha='center', color='C2', fontsize=annotate_size)

axes[1, 1].legend(fontsize=fontsize_legend)
axes[1, 1].set_ylim(-10,165)
axes[1, 1].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Adjust layout
plt.tight_layout()
plt.show()

## Original setup with normalized data error

In [ ]:
# Normalize error
def norm_arr(arr):
    return (arr - arr[-1])/(arr.max() - arr[-1])

In [ ]:
# Note that these data for "iter vs error" are all computed on 80GB A100

# #45368 # path_ptyrad = "H:/workspace/ptyrad/output/paper/tBL_WSe2/20241211_ptyrad_convergence/full_N16384_dp128_flipT100_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.2_ozblur1_oathr0.98_opos_sng1.0_spr0.1_aff1_0_-3_0/model_iter0200.pt"
# #46331 # path_ptyrad = 'H:\workspace\ptyrad\output\paper/tBL_WSe2/20250122_ptyrad_convergence/full_N16384_dp128_flipT100_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_kzf1_sng1.0_aff1_0_-3_0_no_reg/model_iter0200.pt' 
path_ptyrad   = "H:\workspace\ptyrad\output\paper/tBL_WSe2/20250131_ptyrad_convergence/full_N16384_dp128_flipT100_random16_p12_1obj_6slice_dz2_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.5_ozblur1_oathr0.98_opos_sng1.0_spr0.1_aff1_0_-3_0/model_iter0200.pt"
error_ptyrad   = np.array(load_pt(path_ptyrad)['loss_iters'])[:,1]
iter_time_ptyrad = load_pt(path_ptyrad)['avg_iter_t']

# #45367
path_ptyshv = "H:\workspace\ptyrad\data\paper/tBL_WSe2\Panel_g-h_Themis/10/roi10_Ndp128_step128\MLs_ptyrad_p12_g16_pc0_noModel_updW100_mm_Ns6_dz2_reg1_dpFlip_ud_T/Niter200.mat"
with h5py.File(path_ptyshv, "r") as hdf_file:
    error_ptyshv = hdf_file['outputs']['fourier_error_out'][()].squeeze()
    iter_time_ptyshv = hdf_file['outputs']['avgTimePerIter'][()].squeeze()[()]

# #45369
# path_py4dstem = "H:/workspace/ptyrad/output/paper/tBL_WSe2/20241211_py4DSTEM_convergence/N16384_dp128_flipT100_random16_p12_6slice_dz2_update0.1_kzf1_archive/model_iter0200.hdf5"
# #46425, 46426 contains update_step from 0.2 to 1.0 in 0.1 increment
path_py4dstem = "H:\workspace\ptyrad\output\paper/tBL_WSe2/20250124_py4DSTEM_convergence/N16384_dp128_flipT100_random16_p12_6slice_dz2_update0.5_kzf1/model_iter0200.hdf5"
error_py4dstem = load_hdf5(path_py4dstem, dataset_key='error_iterations')
iter_time_py4dstem = load_hdf5(path_py4dstem, dataset_key='iter_times').mean()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.ticker as tck

mpl.rc('xtick', direction='in')
mpl.rc('xtick.major', width=1, size=3.5)
mpl.rc('xtick.minor', width=1, size=2)
mpl.rc('ytick', direction='in')
mpl.rc('ytick.major', width=1, size=3.5)
mpl.rc('ytick.minor', width=1, size=2)

# Plot Data
# Note that these data are all computed on 20GB slice

iterations = np.arange(200) #200
norm_error_ptyrad = norm_arr(error_ptyrad)
norm_error_ptyshv = norm_arr(error_ptyshv)
norm_error_py4dstem = norm_arr(error_py4dstem)

batch_sizes = np.array([16, 32, 64, 128, 256, 512, 1024])
time_ptyrad_batch = np.array([19.0, 16.45, 15.18, 14.37, 13.91, 13.84, 13.71])
time_ptyshv_batch = np.array([84.08, 42.31, 23.47, 17.18, 13.02, 11.87, 11.15])
time_py4dstem_batch = np.array([145.57, 75.85, 43.40, 23.20, 15.47, 12.32, 11.51])

probe_modes = np.array([1, 3, 6, 12])
time_ptyrad_modes = np.array([12.87, 14.18, 19.00, 26.46])
time_ptyshv_modes = np.array([19.07, 45.46, 84.08, 161.49])
time_py4dstem_modes = np.array([29.70, 74.71, 145.57, 282.91])

num_slices = np.array([1, 3, 6])
time_ptyrad_slices = np.array([7.09, 11.34, 19.00])
time_ptyshv_slices = np.array([22.24, 44.38, 84.08])
time_py4dstem_slices = np.array([24.26, 82.01, 145.57])

# Speedup factor
speedup_factor_iter = iter_time_py4dstem/iter_time_ptyrad
speedup_factor_batch = time_py4dstem_batch[0] / time_ptyrad_batch[0]
speedup_factor_modes = time_py4dstem_modes[-1] / time_ptyrad_modes[-1]
speedup_factor_slices = time_py4dstem_slices[-1] / time_ptyrad_slices[-1]

# Global font/line control
linewidth = 0.8
markersize = 4
fontsize_title = 9
fontsize_subtitle = 7
fontsize_label = 9
fontsize_legend = 5

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(7, 5), dpi=300)

# Panel 1: Error vs. Iteration
step = 10  # Select a subset of the data for plotting
axes[0, 0].plot(iterations[::step] * iter_time_ptyrad, norm_error_ptyrad[::step], label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize)
axes[0, 0].plot(iterations[::step] * iter_time_ptyshv, norm_error_ptyshv[::step], label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[0, 0].plot(iterations[::step] * iter_time_py4dstem, norm_error_py4dstem[::step], label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)

axes[0, 0].set_title('Normalized Data Error vs. Reconstruction Time', fontsize=fontsize_title)
axes[0, 0].text(0.45, 0.95, '200 iterations, batch size 16 \n12 probes, 6 slices', 
                transform=axes[0, 0].transAxes, ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[0, 0].set_xlabel('Reconstruction Time (sec)', fontsize=fontsize_label)
axes[0, 0].set_ylabel('Normalized Data Error', fontsize=fontsize_label)
axes[0, 0].text(-0.2, 1.08, 'a', transform=axes[0, 0].transAxes, fontsize=16, fontweight='bold')  # Label "a"
axes[0, 0].text(0.11, 0.125, f'{np.round(speedup_factor_iter,1)}x faster', transform=axes[0, 0].transAxes, fontsize=7, fontweight='bold', c='C0')  # Label "24x faster"
axes[0, 0].legend(fontsize=fontsize_legend)
axes[0, 0].set_ylim(-0.07,1.1)
# axes[0, 0].set_yscale('log')
axes[0, 0].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Panel 2: Iter Time vs. Batch Sizes
axes[0, 1].plot(np.arange(len(batch_sizes)), time_ptyrad_batch, label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize)
axes[0, 1].plot(np.arange(len(batch_sizes)), time_ptyshv_batch, label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[0, 1].plot(np.arange(len(batch_sizes)), time_py4dstem_batch, label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)
axes[0, 1].set_title('Iteration Time vs. Batch Sizes', fontsize=fontsize_title)
axes[0, 1].text(0.5, 0.95, '6 probes, 6 slices', transform=axes[0, 1].transAxes, 
                ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[0, 1].set_xlabel('Batch Sizes', fontsize=fontsize_label)
axes[0, 1].set_ylabel('Iteration Time (sec)', fontsize=fontsize_label)
axes[0, 1].set_xticks(np.arange(len(batch_sizes)))
axes[0, 1].set_xticklabels([str(int(b)) for b in batch_sizes], fontsize=fontsize_label)
axes[0, 1].text(-0.1, 1.08, 'b', transform=axes[0, 1].transAxes, fontsize=16, fontweight='bold')  # Label "b"
axes[0, 1].text(0.01, 0.08, f'{np.round(speedup_factor_batch,1)}x faster', transform=axes[0, 1].transAxes, fontsize=7, fontweight='bold', c='C0')  # Label "7.7x faster"
axes[0, 1].legend(fontsize=fontsize_legend)
axes[0, 1].set_ylim(-10,165)
axes[0, 1].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Panel 3: Iter Time vs. Probe Modes
axes[1, 0].plot(probe_modes, time_ptyrad_modes, label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize)
axes[1, 0].plot(probe_modes, time_ptyshv_modes, label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[1, 0].plot(probe_modes, time_py4dstem_modes, label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)
axes[1, 0].set_title('Iteration Time vs. Probe Modes', fontsize=fontsize_title)
axes[1, 0].text(0.5, 0.95, 'Batch size 16, 6 slices', transform=axes[1, 0].transAxes, 
                ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[1, 0].set_xlabel('Number of Probe Modes', fontsize=fontsize_label)
axes[1, 0].set_ylabel('Iteration Time (sec)', fontsize=fontsize_label)
axes[1, 0].set_xticks(probe_modes)
axes[1, 0].set_xticklabels([str(int(p)) for p in probe_modes], fontsize=fontsize_label)
axes[1, 0].text(-0.2, 1.08, 'c', transform=axes[1, 0].transAxes, fontsize=16, fontweight='bold')  # Label "c"
axes[1, 0].text(0.75, 0.15, f'{np.round(speedup_factor_modes,1)}x faster', transform=axes[1, 0].transAxes, fontsize=7, fontweight='bold', c='C0')  # Label "10.7x faster"
axes[1, 0].legend(fontsize=fontsize_legend)
axes[1, 0].set_ylim(-15,325)
axes[1, 0].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Panel 4: Iter Time vs. Slices
axes[1, 1].plot(num_slices, time_ptyrad_slices, label='PtyRAD', marker='o', linewidth=linewidth, markersize=markersize)
axes[1, 1].plot(num_slices, time_ptyshv_slices, label='PtyShv', marker='s', linewidth=linewidth, markersize=markersize)
axes[1, 1].plot(num_slices, time_py4dstem_slices, label='py4DSTEM', marker='^', linewidth=linewidth, markersize=markersize)
axes[1, 1].set_title('Iteration Time vs. Slices', fontsize=fontsize_title)
axes[1, 1].text(0.5, 0.95, 'Batch size 16, 6 probes', transform=axes[1, 1].transAxes, 
                ha='center', va='top', fontsize=fontsize_subtitle, color='k')
axes[1, 1].set_xlabel('Number of Slices', fontsize=fontsize_label)
axes[1, 1].set_ylabel('Iteration Time (sec)', fontsize=fontsize_label)
axes[1, 1].set_xticks(num_slices)
axes[1, 1].set_xticklabels(num_slices, fontsize=fontsize_label)
axes[1, 1].text(-0.1, 1.08, 'd', transform=axes[1, 1].transAxes, fontsize=16, fontweight='bold')  # Label "d"
axes[1, 1].legend(fontsize=fontsize_legend)
axes[1, 1].set_ylim(-10,165)
axes[1, 1].yaxis.set_minor_locator(tck.AutoMinorLocator())

# Adjust layout
plt.tight_layout()
plt.show()